#Installing important libraries

In [ ]:
!pip install streamlit langchain langchain-google-genai langchain-community pyngrok python-dotenv --quiet faiss-cpu --quiet

#Main Script being written in a file

In [ ]:
from getpass import getpass

# Securing API key as input
gemini_api_key = getpass("🔐 Enter your Gemini API key: ")

# Writing Streamlit code to a file
with open("app.py", "w") as f:
    f.write(f'''
import streamlit as st
import os
from langchain.schema import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings

# Set the Gemini API key
os.environ["GOOGLE_API_KEY"] = "{gemini_api_key}"

# Initialize LLM and embedding
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")
embedding = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")

# Function to load and prepare document data
def load_and_prepare_data(document_text):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
    doc = Document(page_content=document_text)
    document_chunks = text_splitter.split_documents([doc])
    vector_store_faiss = FAISS.from_documents(document_chunks, embedding)
    return vector_store_faiss

# Streamlit UI
st.title('📄 Information Query System (Gemini-Powered)')

uploaded_file = st.file_uploader("Upload a .txt file", type="txt")
if uploaded_file is not None:
    document_text = uploaded_file.read().decode("utf-8")
    vector_store_faiss = load_and_prepare_data(document_text)

    query = st.text_input("Enter your question:")

    if st.button("🔍 Get Answer") and query:
        result = vector_store_faiss.similarity_search(query, k=3)
        context = " ".join([doc.page_content for doc in result])

        prompt = ChatPromptTemplate.from_template(\"\"\"
        Answer the following question using only the context provided.
        <context>
        {{context}}
        </context>
        \"\"\")

        document_chain = create_stuff_documents_chain(llm, prompt)
        retriever = vector_store_faiss.as_retriever()
        retrieval_chain = create_retrieval_chain(retriever, document_chain)

        response = retrieval_chain.invoke({{"input": query, "context": context}})
        answer = response.get('answer', '❌ No answer found.')

        st.markdown("### 🧠 Answer:")
        st.write(answer)
''')

#Using ngrok for creating public URLs for local apps

In [ ]:
from pyngrok import ngrok
import time

authtoken = getpass("🔐 Enter your ngrok authtoken: ")
# Adding authtoken for using pyngrok
!ngrok config add-authtoken {authtoken}

# Kill existing tunnels
ngrok.kill()

# Create public tunnel
public_url = ngrok.connect(8501)
print(f"🚀 Streamlit app is live at: {public_url}")

# Run Streamlit in background
!streamlit run app.py &>/dev/null &
time.sleep(2)  # Wait a bit to make sure the app launches